In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [47]:
# Loading Fani dataset and convert the video into sequence of frames
drive_data_dirs = [
  {
    "link": "/content/drive/My Drive/BronchData/Fani_invivo_data/D97193_crop.avi",
    "video": True,
  },
  #{
  #  "link": "/content/drive/My Drive/ORBDataset/video6/",
  #  "video": False,
  #},
]
data_dir = "/content/data/images/"

import os
os.makedirs(data_dir)

In [46]:
#os.chdir('/content/')
#!rm -rf data

In [48]:
# Load ffmpeg to be able to extract video frames
!apt install ffmpeg

import cv2

def videoToFrames(in_dir, out_dir, global_counter, step=30):
  global_counter += 1
  vidcap = cv2.VideoCapture(in_dir)
  total_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
  success, image = vidcap.read()
  count = 1
  while success:
    cv2.imwrite(out_dir + "frame%d.jpg" % global_counter, image)     # save frame as JPEG file      
    success, image = vidcap.read()
    if count % 1000 == 0:
      print(f'Video frame {count}/{total_frames}: ' + ('completed' if success else 'failed'))
    global_counter += 1
    count += 1
  vidcap.release()
  print('Finished')
  return global_counter

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.6-0ubuntu0.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [49]:
from os import listdir
counter = 0
for dir in drive_data_dirs:
  link = dir["link"]
  print(f"\n Loading data from {link} \n")
  if dir["video"]:
    counter = videoToFrames(link, data_dir,  counter)
  else:
    frames = [f for f in listdir(link)]
    for frame in frames:
      counter += 1
      cv2.imwrite(data_dir + "frame%d.jpg" % counter, cv2.imread(link + frame))


 Loading data from /content/drive/My Drive/BronchData/Fani_invivo_data/D97193_crop.avi 

Video frame 1000/10970: completed
Video frame 2000/10970: completed
Video frame 3000/10970: completed
Video frame 4000/10970: completed
Video frame 5000/10970: completed
Video frame 6000/10970: completed
Video frame 7000/10970: completed
Video frame 8000/10970: completed
Video frame 9000/10970: completed
Video frame 10000/10970: completed
Finished


In [51]:
import os

dir_to_clean = '/content/data/images'
l = os.listdir(dir_to_clean)
print(len(l))

for i, n in enumerate(l):
  if i%100 != 0:
    target = dir_to_clean + '/' + n
    os.remove(target)

10970


In [52]:
l = os.listdir(dir_to_clean)
print(len(l))
print(l[0])

110
frame3727.jpg


**COLMAP**

In [5]:
import os
os.chdir("/content")

In [6]:
!git clone https://github.com/colmap/colmap

Cloning into 'colmap'...
remote: Enumerating objects: 93, done.
remote: Counting objects: 100% (93/93), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 12501 (delta 42), reused 49 (delta 24), pack-reused 12408
Receiving objects: 100% (12501/12501), 13.16 MiB | 19.22 MiB/s, done.
Resolving deltas: 100% (9842/9842), done.


In [7]:
!sudo apt-get install \
    git \
    cmake \
    build-essential \
    libboost-program-options-dev \
    libboost-filesystem-dev \
    libboost-graph-dev \
    libboost-regex-dev \
    libboost-system-dev \
    libboost-test-dev \
    libeigen3-dev \
    libsuitesparse-dev \
    libfreeimage-dev \
    libgoogle-glog-dev \
    libgflags-dev \
    libglew-dev \
    qtbase5-dev \
    libqt5opengl5-dev \
    libcgal-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
libboost-filesystem-dev is already the newest version (1.65.1.0ubuntu1).
libboost-filesystem-dev set to manually installed.
libboost-program-options-dev is already the newest version (1.65.1.0ubuntu1).
libboost-program-options-dev set to manually installed.
libboost-system-dev is already the newest version (1.65.1.0ubuntu1).
libboost-system-dev set to manually installed.
libboost-graph-dev is already the newest version (1.65.1.0ubuntu1).
libboost-graph-dev set to manually installed.
libboost-regex-dev is already the newest version (1.65.1.0ubuntu1).
libboost-regex-dev set to manually installed.
libboost-test-dev is already the newest version (1.65.1.0ubuntu1).
libboost-test-dev set to manually installed.
cmake is already the newest version (3.10.2-1ubuntu2.18.04.1).
git is already the newest version (1:2.17.1-1ubuntu0.7).
qtbase5-d

In [8]:
!sudo apt-get install libcgal-qt5-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  libcgal-qt5-13 libqt5svg5
The following NEW packages will be installed:
  libcgal-qt5-13 libcgal-qt5-dev libqt5svg5
0 upgraded, 3 newly installed, 0 to remove and 35 not upgraded.
Need to get 343 kB of archives.
After this operation, 1,364 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libqt5svg5 amd64 5.9.5-0ubuntu1 [128 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libcgal-qt5-13 amd64 4.11-2build1 [93.9 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libcgal-qt5-dev amd64 4.11-2build1 [122 kB]
Fetched 343 kB in 1s (386 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program 

In [9]:
!sudo apt-get install libatlas-base-dev libsuitesparse-dev
!git clone https://ceres-solver.googlesource.com/ceres-solver
os.chdir("ceres-solver")

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsuitesparse-dev is already the newest version (1:5.1.2-2).
libatlas-base-dev is already the newest version (3.10.3-5).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Cloning into 'ceres-solver'...
remote: Sending approximately 16.12 MiB ...
remote: Counting objects: 29, done
remote: Total 14355 (delta 8241), reused 14355 (delta 8241)
Receiving objects: 100% (14355/14355), 16.12 MiB | 20.06 MiB/s, done.
Resolving deltas: 100% (8241/8241), done.


In [10]:
!git checkout $(git describe --tags)
os.makedirs("build")
os.chdir("build")
!cmake .. -DBUILD_TESTING=OFF -DBUILD_EXAMPLES=OFF
!make
!sudo make install

Note: checking out '1.14.0-280-ge39d9ed'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at e39d9ed Add a missing term and remove a superfluous word
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- De

In [11]:
os.chdir('/content/colmap')

In [12]:
!git checkout dev

Already on 'dev'
Your branch is up to date with 'origin/dev'.


In [13]:
os.makedirs("build")
os.chdir('build')

In [14]:
!cmake ..
!make
!sudo make install

-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Looking for pthread.h
-- Looking for pthread.h - found
-- Looking for pthread_create
-- Looking for pthread_create - not found
-- Looking for pthread_create in pthreads
-- Looking for pthread_create in pthreads - not found
-- Looking for pthread_create in pthread
-- Looking for pthread_create in pthread - found
-- Found Threads: TRUE  
-- Found required Ceres dependency: Eigen version 3.3.4 in /usr/i

In [ ]:
!cp -r /content/drive/My\ Drive/build_ceres-solver /content/ceres-solver/build

In [ ]:
!cp -r /content/drive/My\ Drive/build_colmap /content/colmap/build

In [15]:
os.chdir('/content/colmap/build/src/exe/')
os.getcwd()

'/content/colmap/build/src/exe'

In [16]:
!colmap automatic_reconstructor -h 

COLMAP 3.7 (Commit 527fd03 on 2020-07-24 with CUDA)

Options can either be specified via command-line or by defining
them in a .ini project file passed to `--project_path`.

  -h [ --help ] 
  --random_seed arg (=0)
  --log_to_stderr arg (=0)
  --log_level arg (=2)
  --project_path arg
  --workspace_path arg
  --image_path arg
  --mask_path arg
  --vocab_tree_path arg
  --data_type arg (=individual)       {individual, video, internet}
  --quality arg (=high)               {low, medium, high, extreme}
  --camera_model arg (=SIMPLE_RADIAL)
  --single_camera arg (=0)
  --sparse arg (=1)
  --dense arg (=1)
  --mesher arg (=poisson)             {poisson, delaunay}
  --num_threads arg (=-1)
  --use_gpu arg (=1)
  --gpu_index arg (=-1)



In [53]:
!colmap automatic_reconstructor \
    --workspace_path /content/data \
    --image_path /content/data/images \
    --use_gpu 1


Feature extraction

Processed file [1/110]
  Name:            frame1001.jpg
  Dimensions:      454 x 487
  Camera:          #1 - SIMPLE_RADIAL
  Focal Length:    584.40px
  Features:        122
Processed file [2/110]
  Name:            frame10193.jpg
  Dimensions:      454 x 487
  Camera:          #2 - SIMPLE_RADIAL
  Focal Length:    584.40px
  Features:        310
Processed file [3/110]
  Name:            frame10325.jpg
  Dimensions:      454 x 487
  Camera:          #3 - SIMPLE_RADIAL
  Focal Length:    584.40px
  Features:        51
Processed file [4/110]
  Name:            frame10339.jpg
  Dimensions:      454 x 487
  Camera:          #4 - SIMPLE_RADIAL
  Focal Length:    584.40px
  Features:        38
Processed file [5/110]
  Name:            frame10447.jpg
  Dimensions:      454 x 487
  Camera:          #5 - SIMPLE_RADIAL
  Focal Length:    584.40px
  Features:        108
Processed file [6/110]
  Name:            frame10453.jpg
  Dimensions:      454 x 487
  Camera:          #6

In [ ]:
#!cp -r /content/ceres-solver/build /content/drive/My\ Drive/
#!cp -r /content/colmap/build /content/drive/My\ Drive/

In [54]:
os.chdir('/content/')

In [58]:

!wget https://uajjsw.db.files.1drv.com/y4mX7JZqh1kH_2-cMGnjU6fsKnqgoWNPRC-0V2iaSyhmFRG-ffEckUWEQnM3dEYmlTw-E4jzSeL0Xm30KoshFdeRMTiUtOP5E_3WbCJyYkd10EfGwo2RsqdyaTtmRjLASfbntUEhBs2RtDxihsmMcdAg-qb-qQmstN0Iqxljoxg-BGAxPFEky1j2Y91KmLlTuS4w5wugk7UM2kyDtPQGgoSyw

--2020-07-25 12:50:00--  https://uajjsw.db.files.1drv.com/y4mX7JZqh1kH_2-cMGnjU6fsKnqgoWNPRC-0V2iaSyhmFRG-ffEckUWEQnM3dEYmlTw-E4jzSeL0Xm30KoshFdeRMTiUtOP5E_3WbCJyYkd10EfGwo2RsqdyaTtmRjLASfbntUEhBs2RtDxihsmMcdAg-qb-qQmstN0Iqxljoxg-BGAxPFEky1j2Y91KmLlTuS4w5wugk7UM2kyDtPQGgoSyw
Resolving uajjsw.db.files.1drv.com (uajjsw.db.files.1drv.com)... 13.107.42.12
Connecting to uajjsw.db.files.1drv.com (uajjsw.db.files.1drv.com)|13.107.42.12|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 419421847 (400M) [application/zip]
Saving to: ‘y4mX7JZqh1kH_2-cMGnjU6fsKnqgoWNPRC-0V2iaSyhmFRG-ffEckUWEQnM3dEYmlTw-E4jzSeL0Xm30KoshFdeRMTiUtOP5E_3WbCJyYkd10EfGwo2RsqdyaTtmRjLASfbntUEhBs2RtDxihsmMcdAg-qb-qQmstN0Iqxljoxg-BGAxPFEky1j2Y91KmLlTuS4w5wugk7UM2kyDtPQGgoSyw’

y4mX7JZqh1kH_2-cMGn 100%[===================>] 399.99M  33.3MB/s    in 15s     

2020-07-25 12:50:15 (27.5 MB/s) - ‘y4mX7JZqh1kH_2-cMGnjU6fsKnqgoWNPRC-0V2iaSyhmFRG-ffEckUWEQnM3dEYmlTw-E4jzSeL0Xm30KoshFdeRMTiUtOP5E_3WbCJyYkd10E

In [ ]:
!unzip test_sfm

In [62]:
os.chdir('/content/colmap/build/src/exe/')
os.getcwd()

'/content/colmap/build/src/exe'

In [64]:
!colmap automatic_reconstructor \
    --workspace_path /content/south-building \
    --image_path /content/south-building/images \
    --use_gpu 1

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
PatchMatchOptions
-----------------
max_image_size: 2400
gpu_index: 0
depth_min: 0.756258
depth_max: 3.68575
window_radius: 5
window_step: 1
sigma_spatial: 5
sigma_color: 0.2
num_samples: 15
ncc_sigma: 0.6
min_triangulation_angle: 1
incident_angle_sigma: 0.9
num_iterations: 5
geom_consistency: 0
geom_consistency_regularizer: 0.3
geom_consistency_max_cost: 3
filter: 0
filter_min_ncc: 0.1
filter_min_triangulation_angle: 3
filter_min_num_consistent: 2
filter_geom_consistency_max_cost: 1
write_consistency_graph: 0

PatchMatch::Run
---------------
Initialization: 0.6752s
 Sweep 1: 2.5756s
 Sweep 2: 3.2522s
 Sweep 3: 2.4458s
 Sweep 4: 3.2500s
Iteration 1: 11.5239s
 Sweep 1: 2.4527s
 Sweep 2: 3.2691s
 Sweep 3: 2.4506s
 Sweep 4: 3.2579s
Iteration 2: 11.4306s
 Sweep 1: 2.4449s
 Sweep 2: 3.2545s
 Sweep 3: 2.4290s
 Sweep 4: 3.2356s
Iteration 3: 11.3642s
 Sweep 1: 2.4105s
 Sweep 2: 3.2151s
 Sweep 3: 2.4004s
 Sweep 4: 3.1

In [66]:
!zip -r /content/sparse-building-dense.zip /content/south-building/dense

  adding: content/south-building/dense/ (stored 0%)
  adding: content/south-building/dense/0/ (stored 0%)
  adding: content/south-building/dense/0/stereo/ (stored 0%)
  adding: content/south-building/dense/0/stereo/depth_maps/ (stored 0%)
  adding: content/south-building/dense/0/stereo/depth_maps/P1180313.JPG.photometric.bin (deflated 14%)
  adding: content/south-building/dense/0/stereo/depth_maps/P1180209.JPG.photometric.bin (deflated 21%)
  adding: content/south-building/dense/0/stereo/depth_maps/P1180214.JPG.photometric.bin (deflated 18%)
  adding: content/south-building/dense/0/stereo/depth_maps/P1180314.JPG.photometric.bin (deflated 15%)
  adding: content/south-building/dense/0/stereo/depth_maps/P1180210.JPG.photometric.bin (deflated 22%)
  adding: content/south-building/dense/0/stereo/depth_maps/P1180325.JPG.photometric.bin (deflated 14%)
  adding: content/south-building/dense/0/stereo/depth_maps/P1180203.JPG.photometric.bin (deflated 16%)
  adding: content/south-building/dense/0